<a href="https://colab.research.google.com/github/jpsiegel/Projects/blob/master/ElevatorSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simplified simulation of an elevator transporting people between floors in a building

**To Do:**
- multiple elevators
- add capacity (multiple people per elevator)
- add routing optimization (pickup in between)
- More realistic demand
  - Rush hours
  - More frequency for 1st floor

In [40]:
!pip install simpy --quiet
import simpy

In [45]:
# elevator.py

from collections import deque

DEFAULT_WAIT_TIME = 1.0 # seconds
DEFAULT_SPEED = 1.0 # floors per sec
DEFAULT_LAMBDA = 0.1 # average of requests per second, for poisson process


class Elevator:
    def __init__(self, env: simpy.Environment, floors: tuple[int], speed_floors_per_sec: float, base_floor: int):
        """
        Elevator agent, takes requests and moves across floors.

        Args:
            env: SimPy environment
            floors: Valid floor numbers (e.g., (1, 2, 3, ..., 10))
            speed_floors_per_sec: Constant speed of elevator in floors per second
        """
        self.env = env
        self.floors = floors
        self.speed = speed_floors_per_sec
        self.base_floor = base_floor if base_floor in self.floors else None

        self.current_floor = base_floor
        self.task_queue = deque()
        self.moving = False

        # Start the elevator process
        self.process = env.process(self.run())

    def add_task(self, target_floor: int):
        """
        Enqueue a request to move to a specific floor.
        """
        if target_floor not in self.floors:
            raise ValueError(f"Invalid floor: {target_floor}")
        self.task_queue.append(target_floor)

    def hold(self, duration: float):
        """
        Elevator remains idle at current floor for a set duration.
        """
        yield self.env.timeout(duration)

    def move_to(self, target_floor: int):
        """
        Simulates elevator travel from current floor to target_floor.
        Uses constant speed to compute duration.
        """
        # Calculate travel time
        floor_diff = abs(self.current_floor - target_floor)
        if floor_diff == 0:
            return  # Already at floor
        travel_time = floor_diff / self.speed

        # Move event
        self.moving = True
        print(f"[{self.env.now:.1f}] Elevator starting move from {self.current_floor} to {target_floor}")
        yield self.env.timeout(travel_time)

        self.current_floor = target_floor
        self.moving = False
        print(f"[{self.env.now:.1f}] Elevator arrived at floor {self.current_floor}")

    def run(self):
        """
        Elevator main loop: process queued tasks in FIFO order.
        """
        while True:
            if self.task_queue:

              # Get next task
              next_floor = self.task_queue.popleft()
              print(f"[{self.env.now:.1f}] Elevator processing request to floor {next_floor}")

              # Move if necesary
              if next_floor == self.current_floor and not self.moving:
                print(f"[{self.env.now:.1f}] Elevator is already at floor {next_floor}")
              else:
                yield self.env.process(self.move_to(next_floor))
                yield self.env.process(self.hold(DEFAULT_WAIT_TIME)) # hold briefly after arrival
            else:

              # No tasks, execute resting policy:

              # 1. Stay at current floor
              # yield self.env.timeout(0.5)

              # 2. Go to base floor
              next_floor = self.base_floor

              if self.current_floor != next_floor:
                print(f"[{self.env.now:.1f}] Elevator vacant, going to floor {next_floor}")
                yield self.env.process(self.move_to(next_floor))

              yield self.env.timeout(0.1)




In [46]:
# demandGenerator.py

import simpy
import random

class DemandGenerator:
    def __init__(self, env: simpy.Environment, floors: tuple[int], elevator, lambda_: float):
        """
        Generates elevator demand at random intervals.

        Args:
            env: SimPy environment
            floors: Valid floor numbers
            elevator: Reference to the Elevator instance
            lambda_: Mean arrival interval (Exponential distribution)
        """
        self.env = env
        self.floors = floors
        self.elevator = elevator
        self.lambda_ = lambda_

        # Start the generator process
        self.process = env.process(self.run())

    def generate_interarrival_time(self) -> float:
        """
        Samples the next interarrival time from an exponential distribution.
        """
        return random.expovariate(self.lambda_)

    def generate_origin_destination(self) -> tuple[int, int]:
        """
        Randomly selects origin and destination floors (must differ).
        """
        origin = random.choice(self.floors)
        destination = random.choice([f for f in self.floors if f != origin])
        return origin, destination

    def run(self):
        """
        Main loop: generates demand at stochastic intervals and sends tasks to the elevator.
        """
        while True:
            # Wait until next demand
            interarrival_time = self.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)

            # Generate a random request
            origin, destination = self.generate_origin_destination()

            # Elevator gets a task to go to origin then to destination
            print(f"[{self.env.now:.1f}] Request: from {origin} to {destination}")
            self.elevator.add_task(origin)
            self.elevator.add_task(destination)


In [47]:
# simulation.py

#from elevator import Elevator
#from demand_generator import DemandGenerator

class Simulation:
    def __init__(
        self,
        sim_time: float,
        floors: tuple[int],
        speed_floors_per_sec: float,
        lambda_: float,
        base_floor: int
    ):
        """
        Main simulation controller.

        Args:
            sim_time: Total duration of the simulation (in seconds)
            floors: Valid floor numbers (e.g., (1, 2, ..., 10))
            speed_floors_per_sec: Elevator travel speed
            lambda_: Average time between user requests (Poisson process)
            base_floor: Starting floor and target for resting policy
        """
        self.sim_time = sim_time
        self.env = simpy.Environment()

        # Initialize elevator and demand generator
        self.elevator = Elevator(
            env=self.env,
            floors=floors,
            speed_floors_per_sec=speed_floors_per_sec,
            base_floor=base_floor
        )

        self.demand_generator = DemandGenerator(
            env=self.env,
            floors=floors,
            elevator=self.elevator,
            lambda_=lambda_
        )

    def run(self):
        """
        Runs the simulation.
        """
        self.env.run(until=self.sim_time)


In [50]:
# run

if __name__ == "__main__":
    sim = Simulation(
        sim_time=100,                  # Run for 100 simulated seconds
        floors=tuple(range(1, 6)),     # Floors 1 to 5
        speed_floors_per_sec=DEFAULT_SPEED,
        lambda_=DEFAULT_LAMBDA,
        base_floor=1
    )
    sim.run()


[2.1] Request: from 5 to 3
[2.2] Elevator processing request to floor 5
[2.2] Elevator starting move from 1 to 5
[3.4] Request: from 4 to 2
[6.2] Elevator arrived at floor 5
[7.2] Elevator processing request to floor 3
[7.2] Elevator starting move from 5 to 3
[9.2] Elevator arrived at floor 3
[10.2] Elevator processing request to floor 4
[10.2] Elevator starting move from 3 to 4
[11.2] Elevator arrived at floor 4
[12.2] Elevator processing request to floor 2
[12.2] Elevator starting move from 4 to 2
[14.2] Elevator arrived at floor 2
[15.2] Elevator vacant, going to floor 1
[15.2] Elevator starting move from 2 to 1
[16.2] Elevator arrived at floor 1
[23.4] Request: from 3 to 1
[23.4] Elevator processing request to floor 3
[23.4] Elevator starting move from 1 to 3
[25.4] Elevator arrived at floor 3
[26.4] Elevator processing request to floor 1
[26.4] Elevator starting move from 3 to 1
[28.4] Elevator arrived at floor 1
[45.6] Request: from 1 to 3
[45.7] Elevator processing request to fl

- FORKEAR REPO, empezar a commitear ahi (+ project alfnial)
- pensar stats necesarios
- calcular y logear stats
- guardar eventlog (demand table)
- guardar stats (metadata table)

Opcional
- añadir peak demand a base floor (business logic)
- crear resting policy: por ahora ir a base floor
